In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.model_selection import train_test_split

#Import data
df = pd.read_csv('df_temporal.csv',index_col=0)
df.set_index(['ID','time','t'],drop=True,inplace=True)
df.reset_index(drop=False,inplace=True)
df.fillna(0,inplace=True)
ids = df.ID.unique()

window = 5
t_test = []
t_pred = []

In [2]:
for i in ids:
    X = df[df.ID.values == i].iloc[:,range(3,len(df.columns))]
    
    last = int(len(df[df.ID.values == i])/5)
    X_train = X[:-last]
    X_test = X[-last-window:]
    in_tr = []
    out_tr = []
    for j in range(window,len(X_train)):
        in_tr.append(np.array(X_train.iloc[j-window:j,:]))
        out_tr.append(np.array(X_train.iloc[j,0]))
     
    in_tr, out_tr = np.array(in_tr), np.array(out_tr)

    in_te = []
    out_te = []
    for j in range(window,len(X_test)):
        in_te.append(np.array(X_test.iloc[j-window:j,:]))
        out_te.append(np.array(X_test.iloc[j,0]))
     
    in_te, out_te = np.array(in_te), np.array(out_te)

    model = Sequential()

    model.add(LSTM(128, input_shape=(in_tr.shape[1:]), activation='relu',return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(128, input_shape=(in_tr.shape[1:]), activation='relu',return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1, input_shape=([in_tr.shape[0],128]), activation='sigmoid'))

    model.compile(loss='mse',optimizer='adam')
    model.fit(in_tr, out_tr, epochs=100,verbose=0,batch_size=1)

    t_test.extend(out_te.tolist())
    predictions = model.predict(in_te)
    for pred in range(len(predictions)):
        t_pred.append(predictions.tolist()[pred][0])


In [3]:
from sklearn.metrics import mean_squared_error
MSE=mean_squared_error(t_test,t_pred)
print(MSE)

0.08577518546616705


t_pred = model.predict(in_te)
plt.figure()
plt.plot(t_pred,':',label='LSTM')
plt.plot(t_test,'--',label='Actual')
plt.legend()